In [40]:
import os
import pandas as pd
from glob import glob
from datetime import datetime
import numpy as np

In [22]:
# Read VIX data
vix_data = pd.read_csv("monthly_data.csv")

# Read final settlement data
final_settle_df = pd.read_csv("final_settle_df.csv")

In [23]:
# Ensure 'Expiration' is datetime
final_settle_df['Expiration'] = pd.to_datetime(final_settle_df['Expiration'])

print(final_settle_df.head())

  Product ContractSymbol Duration Type Expiration  SettlementPrice
0      VX     VX/J5_2025       Monthly 2025-04-16            31.91
1      VX     VX/H5_2025       Monthly 2025-03-18            21.63
2      VX     VX/G5_2025       Monthly 2025-02-19            15.94
3      VX     VX/F5_2025       Monthly 2025-01-22            15.53
4      VX     VX/Z4_2024       Monthly 2024-12-18            15.66


In [24]:
# Ensure 'Data' is datetime
vix_data['Date'] = pd.to_datetime(vix_data['Date'])
print(vix_data.head())

        Date  Close   Contract  Year ContractSymbol ContractSymbol1
0 2013-04-22  19.55  CFE_VX_F4  2013          VX/F4      VX/F4_2014
1 2013-04-23  19.35  CFE_VX_F4  2013          VX/F4      VX/F4_2014
2 2013-04-24  19.15  CFE_VX_F4  2013          VX/F4      VX/F4_2014
3 2013-04-25  19.25  CFE_VX_F4  2013          VX/F4      VX/F4_2014
4 2013-04-26  19.14  CFE_VX_F4  2013          VX/F4      VX/F4_2014


In [25]:
# Sort dataframes by date
vix_data = vix_data.sort_values(by='Date')
final_settle_df = final_settle_df.sort_values(by='Expiration')

In [26]:
# Merge this two dataframes on common Contract Symbol
vix_data = pd.merge(
    vix_data,
    final_settle_df[['ContractSymbol', 'Expiration', 'SettlementPrice']],
    left_on='ContractSymbol1',
    right_on='ContractSymbol',
    how='left'
)
print(vix_data)

            Date  Close    Contract  Year ContractSymbol_x ContractSymbol1  \
0     2013-04-22  19.55   CFE_VX_F4  2013            VX/F4      VX/F4_2014   
1     2013-04-23  19.35   CFE_VX_F4  2013            VX/F4      VX/F4_2014   
2     2013-04-24  19.15   CFE_VX_F4  2013            VX/F4      VX/F4_2014   
3     2013-04-25  19.25   CFE_VX_F4  2013            VX/F4      VX/F4_2014   
4     2013-04-26  19.14   CFE_VX_F4  2013            VX/F4      VX/F4_2014   
...          ...    ...         ...   ...              ...             ...   
24760 2024-12-12  14.51  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
24761 2024-12-13  14.55  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
24762 2024-12-16  14.75  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
24763 2024-12-17  15.75  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
24764 2024-12-18  15.40  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   

      ContractSymbol_y Expiration  SettlementPrice  
0         

In [27]:
# Ensure 'Date' is in datetime format
vix_data['Date'] = pd.to_datetime(vix_data['Date'])

# Ensure 'Expiration' is also datetime
vix_data['Expiration'] = pd.to_datetime(vix_data['Expiration'])

# Calculate days to expiry
vix_data['DaysToExpiry'] = (vix_data['Expiration'] - vix_data['Date']).dt.days

In [30]:
vix_data = (
    vix_data[vix_data['DaysToExpiry'] > 0]
    .sort_values(['Date', 'DaysToExpiry'])
    .groupby('Date', group_keys=False)[vix_data.columns]  # explicitly include all columns
    .apply(lambda x: x if (x['DaysToExpiry'].iloc[0] == 1 and len(x) > 1) else x.iloc[[0]])
    .reset_index(drop=True)
)

In [32]:
vix_data = (
    vix_data.sort_values(['Date', 'DaysToExpiry'])
            .groupby('Date', group_keys=False)[vix_data.columns]  # explicitly select all columns
            .apply(lambda x: x.iloc[:2] if x.iloc[0]['DaysToExpiry'] == 1 else x.iloc[[0]])
            .reset_index(drop=True)
)

In [33]:
print(vix_data)

           Date  Close    Contract  Year ContractSymbol_x ContractSymbol1  \
0    2013-04-22  19.55   CFE_VX_F4  2013            VX/F4      VX/F4_2014   
1    2013-04-23  19.35   CFE_VX_F4  2013            VX/F4      VX/F4_2014   
2    2013-04-24  19.15   CFE_VX_F4  2013            VX/F4      VX/F4_2014   
3    2013-04-25  19.25   CFE_VX_F4  2013            VX/F4      VX/F4_2014   
4    2013-04-26  19.14   CFE_VX_F4  2013            VX/F4      VX/F4_2014   
...         ...    ...         ...   ...              ...             ...   
3064 2024-12-11  14.36  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
3065 2024-12-12  14.51  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
3066 2024-12-13  14.55  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
3067 2024-12-16  14.75  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
3068 2024-12-17  15.75  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   

     ContractSymbol_y Expiration  SettlementPrice  DaysToExpiry  
0        

In [34]:
vix_data = vix_data[vix_data['Date'] > '2014-01-20']
vix_data = vix_data[vix_data['DaysToExpiry'] >=1]

In [42]:
vix_data = vix_data.set_index('Date')
# Next contract (DaysToExpiry > 0), pick the one with the smallest DaysToExpiry > 0 per date
next_contract = (
    vix_data[vix_data['DaysToExpiry'] > 1]
    .sort_values('DaysToExpiry')  # Ensure the soonest next contract is chosen
    .groupby(level=0)  # group by Date (index)
    .first()[['Close']]
    .rename(columns={'Close': 'Close_next'})
)

expiring = vix_data[vix_data['DaysToExpiry'] == 1].copy()
# Join both into one row per date
data_rolled = expiring.join(next_contract, how='left')
next_contract.name = "price_next"
print(next_contract)

            Close_next
Date                  
2014-01-21       14.06
2014-01-22       13.82
2014-01-23       14.17
2014-01-24       16.21
2014-01-27       16.09
...                ...
2024-12-10       14.56
2024-12-11       14.36
2024-12-12       14.51
2024-12-13       14.55
2024-12-16       14.75

[2748 rows x 1 columns]


In [43]:
df = vix_data.merge(next_contract, left_index=True, right_index=True, how='left')
# Keep Close_next only for rows where DaysToExpiry == 1
df['Close_next'] = df.apply(
    lambda row: row['Close_next'] if row['DaysToExpiry'] == 1 else np.nan,
    axis=1
)
df = df.reset_index()  # If Date is in the index, move it to a column
df = df.drop_duplicates(subset=['Date'], keep='first')
df = df.set_index('Date')  # Optional: set Date back as index if needed
print(df)

            Close    Contract  Year ContractSymbol_x ContractSymbol1  \
Date                                                                   
2014-01-21  13.23   CFE_VX_F4  2014            VX/F4      VX/F4_2014   
2014-01-22  13.82   CFE_VX_G4  2014            VX/G4      VX/G4_2014   
2014-01-23  14.17   CFE_VX_G4  2014            VX/G4      VX/G4_2014   
2014-01-24  16.21   CFE_VX_G4  2014            VX/G4      VX/G4_2014   
2014-01-27  16.09   CFE_VX_G4  2014            VX/G4      VX/G4_2014   
...           ...         ...   ...              ...             ...   
2024-12-11  14.36  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
2024-12-12  14.51  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
2024-12-13  14.55  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
2024-12-16  14.75  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   
2024-12-17  15.75  CFE_VX_Z24  2024           VX/Z24      VX/Z4_2024   

           ContractSymbol_y Expiration  SettlementPrice  DaysTo

In [44]:
df.to_csv("data_vix.csv", index=True)